# Project Template App - Ngrok, Colab and Yolo

In [1]:
!pip install flask-ngrok > /dev/null 2>&1
!pip install pyngrok > /dev/null 2>&1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
with open('drive/MyDrive/project/static/ngrok_auth.txt') as nf:
    ngrok_auth = nf.read()

In [4]:
!ngrok authtoken $ngrok_auth

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
!git clone https://github.com/ultralytics/yolov5 > /dev/null 2>&1

In [6]:
%cd yolov5
!pip install -r requirements.txt > /dev/null 2>&1

/content/yolov5


In [7]:
%cd /content
!cp /content/drive/MyDrive/project/yolov5/runs/train/gun_yolov5s_results/weights/best.pt .

/content


### Web Server Code

In [8]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import os, subprocess, shutil

app = Flask(__name__, static_folder='/content/drive/MyDrive/project/static',
                      template_folder='/content/drive/MyDrive/project/templates')
run_with_ngrok(app)

@app.route('/')
def home():
    menu = {'home':1, 'menu':0, 'yolo':0}
    return render_template('index.html', menu=menu)

@app.route('/menu', methods=['GET','POST'])
def menu():
    menu = {'home':0, 'menu':1, 'yolo':0}
    if request.method == 'GET':
        languages = [
            {'disp':'영어', 'val':'en'},
            {'disp':'일어', 'val':'jp'},
            {'disp':'중국어', 'val':'cn'},
            {'disp':'프랑스어', 'val':'fr'},
            {'disp':'스페인어', 'val':'es'}
        ]
        return render_template('menu.html', menu=menu,
                                options=languages)   # 서버에서 클라이언트로 정보 전달
    else:
        # 사용자가 입력한 정보를 서버가 읽음
        index = request.form['index']
        lang = request.form['lang']
        lyrics = request.form['lyrics']
        #print(lang, '\n', index, '\n', lyrics, sep='')
        # 사용자가 입력한 파일을 읽어서 upload 디렉토리에 저장
        f_image = request.files['image']
        fname = f_image.filename                # 사용자가 입력한 파일 이름
        filename = os.path.join(app.static_folder, 'upload/') + fname
        f_image.save(filename)

        # 모델 실행후 결과를 돌려줌
        result = '독수리 (73.52%)'
        mtime = int(os.stat(filename).st_mtime)
        return render_template('menu_res.html', result=result, menu=menu,
                                fname=fname, mtime=mtime)

@app.route('/yolo', methods=['GET','POST'])
def yolo():
    menu = {'home':0, 'menu':0, 'yolo':1}
    if request.method == 'GET':
        return render_template('yolo.html', menu=menu)
    else:
        # 사용자가 입력한 파일을 읽어서 upload 디렉토리에 저장
        f_image = request.files['image']
        fname = f_image.filename                # 사용자가 입력한 파일 이름
        filename = os.path.join(app.static_folder, 'upload/') + fname
        f_image.save(filename)
        fext = fname.split('.')[-1]

        cmd = f'cd yolov5; python detect.py --weights ../best.pt --img 416 --conf 0.5 --source {filename}'
        r = subprocess.run(cmd, shell=True, capture_output=True)
        res_file = os.path.join(app.static_folder, 'upload/yolo_img.' + fext)
        if os.path.exists('/content/yolov5/runs/detect/exp2'):
            shutil.copy('/content/yolov5/runs/detect/exp2/'+fname, res_file)
            shutil.rmtree('/content/yolov5/runs/detect/exp2')
        else:
            shutil.copy('/content/yolov5/runs/detect/exp/'+fname, res_file)

        mtime = int(os.stat(res_file).st_mtime)
        return render_template('yolo_res.html', menu=menu, fext=fext, mtime=mtime)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4187-34-91-40-136.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 06:44:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 06:44:03] "GET /static/img/logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 06:44:04] "GET /static/img/favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Aug/2022 06:44:06] "GET /yolo HTTP/1.1" 200 -
ERROR:__main__:Exception on /yolo [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full